# 如何进行工具/函数调用

:::info
我们将工具调用与函数调用互换使用。尽管函数调用有时指的是单个函数的调用，但我们对待所有模型的方式是，它们可以在每次消息中返回多个工具或函数调用。
:::

工具调用允许模型通过生成与用户定义的模式匹配的输出来响应给定提示。虽然名称暗示模型正在执行某些操作，但事实并非如此！模型只是想出了一个工具的参数，而实际运行该工具（或不运行）取决于用户——例如，如果您想从非结构化文本中[提取与某些模式匹配的输出](/docs/tutorials/extraction)，您可以给模型一个“提取”工具，该工具接受匹配所需模式的参数，然后将生成的输出视为最终结果。

工具调用包含一个名称、一个参数字典和一个可选的标识符。参数字典的结构为 `{参数名: 参数值}`。

包括 [Anthropic](https://www.anthropic.com/)、[Cohere](https://cohere.com/)、[Google](https://cloud.google.com/vertex-ai)、[Mistral](https://mistral.ai/)、[OpenAI](https://openai.com/) 等在内的许多 LLM 提供商都支持工具调用功能的变体。这些功能通常允许发送给 LLM 的请求包含可用的工具及其模式，并允许响应包含对这些工具的调用。例如，给定一个搜索引擎工具，LLM 可能会先发出一个对该搜索引擎的调用来处理查询。调用 LLM 的系统可以接收工具调用，执行它，然后将输出返回给 LLM 以告知其响应。LangChain 包含一套[内置工具](/docs/integrations/tools/)，并支持多种定义您自己的[自定义工具](/docs/how_to/custom_tools)的方法。工具调用对于构建[使用工具的链和代理](/docs/how_to#tools)以及更普遍地从模型中获取结构化输出来说都极其有用。

提供商在格式化工具模式和工具调用方面采用了不同的约定。例如，Anthropic 将工具调用作为解析后的结构返回到更大的内容块中：
```python
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]
```
而 OpenAI 则将工具调用分离到一个单独的参数中，参数是 JSON 字符串：
```python
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}
```
LangChain 实现了定义工具、将它们传递给 LLM 以及表示工具调用的标准接口。

## 将工具传递给 LLM

支持工具调用功能的聊天模型实现了一个 `.bind_tools` 方法，该方法接收一个 LangChain [工具对象](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool)列表，并将它们以其预期的格式绑定到聊天模型。随后对聊天模型的调用将在其对 LLM 的调用中包含工具模式。

例如，我们可以使用 Python 函数上的 `@tool` 装饰器来定义自定义工具的模式：

In [22]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

或者，我们在下面使用 Pydantic 定义 schema：

In [23]:
from pydantic import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

我们可以按如下方式将它们绑定到聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
  overrideParams={{fireworks: {model: "accounts/fireworks/models/firefunction-v1", kwargs: "temperature=0"}}}
/>

我们可以使用 `bind_tools()` 方法来处理将 `Multiply` 转换为“工具”并将其绑定到模型（即，每次调用模型时传递它）。

In [67]:
# | echo: false
# | output: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [68]:
llm_with_tools = llm.bind_tools(tools)

## 工具调用

如果语言模型响应中包含工具调用，它们将作为一系列 `tool call` 对象附加到相应的 [message](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage) 或 [message chunk](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk) 的 `.tool_calls` 属性中。`ToolCall` 是一个类型化的字典，包括工具名称、参数值字典以及（可选的）标识符。没有工具调用的消息在此属性上默认为空列表。

示例：

In [15]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_1Tdp5wUXbYQzpkBoagGXqUTo'},
 {'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_k9v09vYioS3X0Qg35zESuUKI'}]

`.tool_calls` 属性应包含有效的工具调用。请注意，有时模型提供程序可能会输出格式错误的工具调用（例如，不是有效 JSON 的参数）。在这种情况下，如果解析失败，`.invalid_tool_calls` 属性将填充 `InvalidToolCall` 实例。`InvalidToolCall` 可以包含名称、字符串参数、标识符和错误消息。

如果需要，[输出解析器](/docs/how_to#output-parsers) 可以进一步处理输出。例如，我们可以将其转换回原始 Pydantic 类：

In [16]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
chain.invoke(query)

[Multiply(a=3, b=12), Add(a=11, b=49)]

### 流式传输

当在流式传输的上下文中调用工具时，[消息块](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk) 将通过 `.tool_call_chunks` 属性的列表填充具有 [工具调用块](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolCallChunk.html#langchain_core.messages.tool.ToolCallChunk) 对象的属性。`ToolCallChunk` 包含工具的 `name`、`args` 和 `id` 的可选字符串字段，以及一个可选的整数字段 `index`，可用于将块连接在一起。字段是可选的，因为工具调用的部分可能在不同的块之间流式传输（例如，包含参数子字符串的块可能对工具名称和 ID 具有空值）。

由于消息块继承自其父消息类，因此包含工具调用块的 [AIMessageChunk](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk) 还将包含 `.tool_calls` 和 `.invalid_tool_calls` 字段。这些字段是从消息的工具调用块中尽力解析的。

请注意，并非所有提供商目前都支持工具调用的流式传输。

示例：

In [17]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'Multiply', 'args': '', 'id': 'call_d39MsxKM5cmeGJOoYKdGBgzc', 'index': 0}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0}]
[{'name': 'Add', 'args': '', 'id': 'call_QJpdxD9AehKbdXzMHxgDMMhs', 'index': 1}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1}]
[]


值得注意的是，添加 message chunks 会合并它们对应的 tool call chunks。这也是 LangChain 的各种 [tool output parsers](/docs/how_to/output_parser_structured) 支持流式输出的原理。

例如，下面我们累积了 tool call chunks：

In [18]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

[]
[{'name': 'Multiply', 'args': '', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a"', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, ', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}, {'name': 'Add', 'args': '', 'id': 'call_tYHYdEV2YBvzDcSCiFCExNvw', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}, {'name': 'Add', 'args': '{"a"', 'id': 'call_tYHYdEV2YBvzDcSCiFCExNvw', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}, {'name': 'Add', 'args': '{"a": 11,', 'id': 'call_

In [19]:
print(type(gathered.tool_call_chunks[0]["args"]))

<class 'str'>


下面我们累积工具调用来演示部分解析：

In [20]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

[]
[]
[{'name': 'Multiply', 'args': {}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 1}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, {'name': 'Add', 'args': {}, 'id': 'call_UjSHJKROSAw2BDc8cp9cSv4i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_UjSHJKROSAw2BDc8cp9cSv4i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_UjSHJKROSAw2BDc8cp9cSv4i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, 

In [21]:
print(type(gathered.tool_calls[0]["args"]))

<class 'dict'>


## 將工具輸出傳遞給模型

如果我們使用模型生成的工具調用來實際調用工具，並希望將工具結果傳遞回模型，我們可以使用 `ToolMessage` 來實現。

In [117]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_K5DsWEmgt6D08EI9AFu9NaL1', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_qywVrsplg0ZMv7LHYYMjyG81', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1a0b8cdd-9221-4d94-b2ed-5701f67ce9fe-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_K5DsWEmgt6D08EI9AFu9NaL1'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_qywVrsplg0ZMv7LHYYMjyG81'}]),
 ToolMessage(content='36', tool_call_id='call_K5DsWEmgt6D08EI9AFu9NaL1'),
 ToolMessage(content='60', tool_call_id='call_qywVrsplg0ZMv7LHYYMjyG81')]

In [118]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 is 36 and 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 171, 'total_tokens': 189}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6c8093c-b16a-4c92-8308-7c9ac998118c-0')

## 少样本提示 (Few-shot prompting)

对于更复杂的工具使用，在提示中添加少样本示例非常有用。我们可以通过向提示中添加带有 `ToolCall` 和相应 `ToolMessage` 的 `AIMessage` 来实现这一点。

例如，即使有一些特殊的指令，我们的模型在处理运算顺序时也可能出错：

In [112]:
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_Dl3FXRVkQCFW4sUNYOe4rFr7'},
 {'name': 'Add',
  'args': {'a': 952, 'b': -20},
  'id': 'call_n03l4hmka7VZTCiP387Wud2C'}]

模型此时不应尝试添加任何内容，因为它在技术上还不知道 119 * 8 的结果。

通过添加包含一些示例的提示，我们可以纠正此行为：

In [107]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_MoSgwzIhPxhclfygkYaKIsGZ'}]

这次看起来我们得到了正确的输出。

[LangSmith trace](https://smith.langchain.com/public/f70550a1-585f-4c9d-a643-13148ab1616f/r) 看起来是这样的。

## 后续步骤

- **输出解析**: 请参阅 [OpenAI 工具输出解析器](/docs/how_to/output_parser_structured)，了解如何将函数调用 API 响应提取到各种格式。
- **结构化输出链**: [某些模型提供了构造函数](/docs/how_to/structured_output)，可以为您处理创建结构化输出链。
- **工具使用**: 在[这些指南](/docs/how_to#tools)中了解如何构建调用已调用工具的链和代理。